In [4]:
import pandas as pd

In [5]:
df_genome_scores = pd.read_csv('genome-scores.csv')
df_genome_tags = pd.read_csv('genome-tags.csv')
df_link = pd.read_csv('links.csv')
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')
df_tags = pd.read_csv('tags.csv')

In [12]:
df_genome_scores.iloc[44123]

movieId       40.00
tagId        132.00
relevance      0.07
Name: 44123, dtype: float64

In [19]:
df_movies[df_movies['movieId']==50]

,movieId,title,genres
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:
# Pivot the genome scores to create a feature matrix
df_features = df_genome_scores.pivot(index='movieId', columns='tagId', values='relevance').fillna(0)
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity matrix for movies
cosine_sim = cosine_similarity(df_features)


In [10]:
len(df_genome_scores['movieId'].unique())

13816

In [19]:
import pickle
# Load the trained SVD model
with open('svd_model.pkl', 'rb') as f:
    svd = pickle.load(f)


In [1]:
def hybrid_recommendation(user_id, liked_movies, top_n=10):
    # Collaborative Filtering Recommendations
    user_rated_movies = df_ratings[df_ratings['userId'] == user_id]['movieId'].tolist()
    predictions = [svd.predict(user_id, mid) for mid in df_features.index if mid not in user_rated_movies]
    cf_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
    cf_movie_ids = [pred.iid for pred in cf_recommendations]

    # Content-Based Filtering Recommendations
    liked_movie_ids = [df_movies[df_movies['title'] == movie]['movieId'].values[0] for movie in liked_movies]
    liked_movie_indices = [df_features.index.get_loc(mid) for mid in liked_movie_ids]
    content_sim_scores = cosine_sim[liked_movie_indices].mean(axis=0)
    content_movie_indices = content_sim_scores.argsort()[::-1][:top_n]
    
    content_movie_ids = [df_features.index[idx] for idx in content_movie_indices]

    # Combine Recommendations
    combined_movie_ids = list(set(cf_movie_ids + content_movie_ids))[:top_n]
    recommended_movies = [df_movies[df_movies['movieId'] == mid]['title'].values[0] for mid in combined_movie_ids]
    
    return recommended_movies

# Example usage
user_id = 1
liked_movies = ['Toy Story (1995)', 'Jumanji (1995)']
print(hybrid_recommendation(user_id, liked_movies, top_n=10))


NameError: name 'df_ratings' is not defined

In [1]:
import secrets
secret_key = secrets.token_hex(24)
print(secret_key)

72d373c2256e074a908498b1c7bbd85f5a5a603ff233d451
